# Advanced Concepts
In this notebook, I will implement the most used advanced concepts in linear algebra.
## Table of Contents
1. Preliminary concepts
  1. Gaussian Elimination
  2. Projections
2. Matrix Decomposition
  1. LU Decomposition
  2. QR Decomposition
  3. Eigen Decomposition
  4. Singular Value Decomposition
3. Matrix Approximation
  1. Best rank-k approximation with SVD

In [1]:
# imports
import numpy as np

## Preliminary Concepts

### Gaussian Elimination
Gaussian elimination is used to solve the system of linear equations. We use matrix for this.

Where it is used?

To convert a image into 3D model and this is explained in [Immersive Linear Algebra book](https://immersivemath.com/ila/ch05_gausselim/ch05.html)

#### Row Echelon Form
$$\begin{bmatrix} p_1 & a & b \\ 0 & p_2 & c \\ 0 & 0 & p_3 \end{bmatrix}$$
This is the final result of Gaussian Elimination. Where `p` are pivotal elements. The resulted matrix is a Upper triangular matrix.

In [ ]:
def gaussian_elimination(A:np.array) -> np.array:
  ''' Gaussian Elimination with partial pivoting

  :param A:np.array - Matrix with system of linear equation
  :return: np.array - Matrix with Row Echelon Form
  '''
  num_of_rows = A.shape[0]
  for row_index in range(num_of_rows):
    pivotal_row = np.argmax(np.abs(A[row_index:, row_index])) + row_index # taking the maximum pivotal row
    A[[row_index, pivotal_row]] = A[[pivotal_row, row_index]] # swapping the rows

    # elimination process
    for col_index in range(row_index+1,num_of_rows):
      factor = A[col_index,row_index] / A[row_index,row_index]
      A[col_index, row_index:] -= factor * A[row_index, row_index:]

  return A

In [ ]:
A = np.array([
        [3, 2, -1, 1],
        [2, -2, 4, -2],
        [-1, 0.5, -1, 0]
    ], dtype=np.float32)
gaussian_elimination(A)

array([[ 3.        ,  2.        , -1.        ,  1.        ],
       [ 0.        , -3.3333335 ,  4.6666665 , -2.6666667 ],
       [ 0.        ,  0.        ,  0.29999983, -0.6       ]],
      dtype=float32)

### Projections

Projections are used to reduce the multi-dimensional vectors into lower dimensional vectors. It is used in **PCA**.

### Example
$$\vec{a}=\begin{bmatrix}1 \\ 3\end{bmatrix}
\text{   }\vec{b} = \begin{bmatrix}3 \\ 2 \end{bmatrix}$$
When $\vec{a}$ vector is projected on $\vec{b}$ the 2D space is changed to 1D space.

The output of the projection of a on b is $\vec{p} = \begin{bmatrix}2.0 \\ 1.4 \end{bmatrix}$

In [ ]:
def projections(base : np.array, x : np.array) -> np.array :
  projection_matrix = (base @ base.T) / (base.T @ base)
  return projection_matrix @ x

In [ ]:
base = np.array([[3],
              [2]])
x = np.array([[1],
              [3]])
p = projections(base,x)
p

array([[2.07692308],
       [1.38461538]])

## Matrix Decomposition

### LU Decomposition
Formula for the LU Decomposition is $$A=LU$$
where,
* A - A is a square matrix
* L - Lower Triangular matrix
* U - Upper Triangular matrix
#### Algorithm
The LU decomposition works by transforming the original matrix A into a product of two triangular matrices, L and U. The process involves the following steps:
1. Initialize the L and U matrices with zeros.
2. Iterate through each element of the A matrix, starting from the top-left corner.
3. For each element, calculate the corresponding elements in the L and U matrices using the following formulas:
4. For the L matrix: `L[i, j] = A[i, j] - sum(L[i, k] * U[k, j]) for k < j`
5. For the U matrix: `U[i, j] = A[i, j] - sum(L[i, k] * U[k, j]) for k < i`
6. Repeat steps 2-3 until all elements of the A matrix have been processed.

In [ ]:
def LU(A:np.array, verify : bool = False) -> (np.array,np.array):
  '''
  LU is a function that converts a square matrix into two matrix they are lower triangular matrix and upper triangular matrix

  :param A: np.array - Matrix to be decomposed
  :param verify: bool - to print the verification of the function
  :return: (np.array,np.array) - Lower triangular matrix and upper triangular matrix'''

  num_of_rows = A.shape[0]

  # Initializing L and U
  L = np.zeros((num_of_rows,num_of_rows))
  U = np.copy(A).astype(np.float64)

  # making all the diagonal matrix of L to 1
  np.fill_diagonal(L,1)

  # decomposition
  for col_index in range(num_of_rows):
    for element_index in range(col_index+1,num_of_rows):
      L[element_index,col_index] = U[element_index,col_index] / U[col_index,col_index]
      U[element_index,:] -= L[element_index,col_index] * U[col_index,:]

  if verify:
    print("Verification")
    print(f"Given matrix A : \n{A}")
    print(f"Lower Triangular matrix L : \n{L}")
    print(f"Upper Triangular matrix U : \n{U}")
    print(f"L @ U : \n{np.allclose(A,np.dot(L,U))}")

  return L,U

In [ ]:
A = np.array([[2, 1, 1, 0],
              [4, 3, 3, 1],
              [8, 7, 9, 5],
              [6, 7, 9, 8]])

LU(A,verify=True)

Verification
Given matrix A : 
[[2 1 1 0]
 [4 3 3 1]
 [8 7 9 5]
 [6 7 9 8]]
Lower Triangular matrix L : 
[[1. 0. 0. 0.]
 [2. 1. 0. 0.]
 [4. 3. 1. 0.]
 [3. 4. 1. 1.]]
Upper Triangular matrix U : 
[[2. 1. 1. 0.]
 [0. 1. 1. 1.]
 [0. 0. 2. 2.]
 [0. 0. 0. 2.]]
L @ U : 
True


(array([[1., 0., 0., 0.],
        [2., 1., 0., 0.],
        [4., 3., 1., 0.],
        [3., 4., 1., 1.]]),
 array([[2., 1., 1., 0.],
        [0., 1., 1., 1.],
        [0., 0., 2., 2.],
        [0., 0., 0., 2.]]))

### QR Decomposition
This decomposition is useful in solving systems of linear equations, finding the inverse of a matrix, and computing determinants.
The formula : $$A=QR$$
where
* A - matrix
* Q - Orthogonal matrix
* R - Upper Triangular matrix
#### Algorithm
The QR decomposition algorithm involves the following steps:
1. Initialize the Q and R matrices.
2. Apply the `Gram-Schmidt process` to the columns of A to obtain the orthogonal matrix Q.
3. Compute the R matrix using the formula: $R = Q^T A$.

In [ ]:
def QR(A : np.array, verify : bool = False) -> (np.array,np.array):
  '''QR Decomposition is to solve the system of linear equations

  :param A: np.array - Matrix to be decomposed
  :param verify: bool - to print the verification of the function
  :return: (np.array,np.array) - Orthogonal matrix and Upper Triangular matrix'''
  num_rows,num_cols = A.shape

  Q = np.zeros((num_rows,num_cols))
  R = np.zeros((num_rows,num_cols))

  # Gram-Schmidt
  for col in range(num_cols):
    vector = A[:,col]
    # Subtract the projection of v onto the previous columns of Q
    for prev_col in range(col):
      prev_col_elements = Q[:,prev_col]
      R[prev_col,col] = np.dot(prev_col_elements,vector)
      vector = vector - R[prev_col,col] * prev_col_elements
    # normalizing the vector
    norm = np.linalg.norm(vector)
    Q[:,col] = vector / norm
    R[col,col] = norm

  if verify:
    print("Verification")
    print(f"Given matrix A : \n{A}")
    print(f"Orthogonal matrix Q : \n{Q}")
    print(f"Upper Triangular matrix R : \n{R}")
    print(f"L @ U : \n{np.allclose(A,np.dot(Q,R))}")
  return Q,R

In [ ]:
A = np.array([[2, 1, 1, 0],
              [4, 3, 3, 1],
              [8, 7, 9, 5],
              [6, 7, 9, 8]])

QR(A,verify=True)

Verification
Given matrix A : 
[[2 1 1 0]
 [4 3 3 1]
 [8 7 9 5]
 [6 7 9 8]]
Orthogonal matrix Q : 
[[ 0.18257419 -0.46547467  0.          0.8660254 ]
 [ 0.36514837 -0.39386318 -0.79259392 -0.28867513]
 [ 0.73029674 -0.25064021  0.56613852 -0.28867513]
 [ 0.54772256  0.75192062 -0.22645541  0.28867513]]
Upper Triangular matrix R : 
[[10.95445115 10.22415441 12.78019301  8.39841255]
 [ 0.          1.86189867  2.8644595   4.36830073]
 [ 0.          0.          0.67936622  0.22645541]
 [ 0.          0.          0.          0.57735027]]
L @ U : 
True


(array([[ 0.18257419, -0.46547467,  0.        ,  0.8660254 ],
        [ 0.36514837, -0.39386318, -0.79259392, -0.28867513],
        [ 0.73029674, -0.25064021,  0.56613852, -0.28867513],
        [ 0.54772256,  0.75192062, -0.22645541,  0.28867513]]),
 array([[10.95445115, 10.22415441, 12.78019301,  8.39841255],
        [ 0.        ,  1.86189867,  2.8644595 ,  4.36830073],
        [ 0.        ,  0.        ,  0.67936622,  0.22645541],
        [ 0.        ,  0.        ,  0.        ,  0.57735027]]))

### Eigen decomposition
Eigen decomposition is a `factorization` technique used to decompose a square matrix into the product of three matrices:
1. a matrix of eigenvectors,
2. a diagonal matrix of eigenvalues, and
3. the inverse of the eigenvector matrix.

This decomposition is useful in solving systems of linear differential equations, analyzing the stability of systems, and understanding the properties of matrices.

The formula : $$A=VΛV^{-1}$$
where,
* V is a matrix of eigenvectors,
* Λ is a diagonal matrix of eigenvalue
* $V^{-1}$ is the inverse of the eigenvector matrix.

In [ ]:
def eigen_decomposition(A : np.array , verify : bool = False) -> (np.array,np.array,np.array):
  '''
  Eigen Decomposition is used to solve system of linear equation

  :param A: np.array - matrix to be decomposed
  :param verify: bool - to print the verification of the function
  :returns: (np.array,np.array,np.array) - Matrix of eigenvectors, Diagonal matrix of eigen value and Inverse of Eigenvectors
  '''
  eigenvalues,eigenvectors = np.linalg.eig(A)
  lambda_matrix = np.diag(eigenvalues)
  inverse_eignevectors = np.linalg.inv(eigenvectors)

  if verify:
    print("Verification")
    print(f"Given matrix A : \n{A}")
    print(f"Matrix of eigenvectors : \n{eigenvectors}")
    print(f"Diagonal matrix of eigen value : \n{lambda_matrix}")
    print(f"Inverse of Eigenvectors : \n{inverse_eignevectors}")
    print(f"V @ Λ @ V^-1 : \n{np.allclose(A,np.dot(np.dot(eigenvectors,lambda_matrix),inverse_eignevectors))}")

  return eigenvectors,lambda_matrix,inverse_eignevectors

In [ ]:
A = np.array([[2, 1, 1, 0],
              [4, 3, 3, 1],
              [8, 7, 9, 5],
              [6, 7, 9, 8]])

eigen_decomposition(A,True)

Verification
Given matrix A : 
[[2 1 1 0]
 [4 3 3 1]
 [8 7 9 5]
 [6 7 9 8]]
Matrix of eigenvectors : 
[[ 0.05217777  0.24958023  0.21781155 -0.39040915]
 [ 0.19290936  0.34361034  0.46497954  0.86229217]
 [ 0.62976442  0.02230173 -0.78847685 -0.17999628]
 [ 0.75063988 -0.9050659   0.33860952 -0.26764575]]
Diagonal matrix of eigen value : 
[[17.76674972  0.          0.          0.        ]
 [ 0.          3.46610996  0.          0.        ]
 [ 0.          0.          0.51478349  0.        ]
 [ 0.          0.          0.          0.25235682]]
Inverse of Eigenvectors : 
[[ 0.73358274  0.63864187  0.77850317  0.46393712]
 [ 1.27998936  0.49175105  0.40513445 -0.55524578]
 [ 0.88496413  0.38386049 -0.63678527  0.37407947]
 [-1.1513763   0.61387776  0.00777369 -0.08425094]]
V @ Λ @ V^-1 : 
True


(array([[ 0.05217777,  0.24958023,  0.21781155, -0.39040915],
        [ 0.19290936,  0.34361034,  0.46497954,  0.86229217],
        [ 0.62976442,  0.02230173, -0.78847685, -0.17999628],
        [ 0.75063988, -0.9050659 ,  0.33860952, -0.26764575]]),
 array([[17.76674972,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  3.46610996,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.51478349,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.25235682]]),
 array([[ 0.73358274,  0.63864187,  0.77850317,  0.46393712],
        [ 1.27998936,  0.49175105,  0.40513445, -0.55524578],
        [ 0.88496413,  0.38386049, -0.63678527,  0.37407947],
        [-1.1513763 ,  0.61387776,  0.00777369, -0.08425094]]))

### SVD
In eigen decomposition, the matrix `A` must be square matrix. This is the major problem and to solve this issue, SVD is introduced. SVD splits `A` into :    
1. Orthogonal matrix that contains left singular vectors
2. Diagonal matrix of singular values
3. Orthogonal matrix that contains right singular vectors

The formula : $$A = UΣV^T$$
where,
* A - can be any matrix
* U - Orthogonal Matrix
* $V^T$ - Transpose of Orthogonal matrix that is same as in Eigen decomposition
* $Σ$ - Diagonal matrix of singular values
#### Algorithm
1. Calculate the matrix A^T A, where A^T is the transpose of A.
2. Calculate the eigenvalues and eigenvectors of A^T A.
3. The eigenvalues of A^T A are the squares of the singular values of A.
4. The eigenvectors of A^T A are the right singular vectors of A.
5. Calculate the left singular vectors of A by multiplying A with the right singular vectors and dividing by the corresponding singular values.

#### Resources
1. [Visualization of SVD - YouTube](https://youtu.be/vSczTbgc8Rc?si=J74qCUlHzuCOjWt_)

In [2]:
def SVD(A:np.array,verify : bool = False) -> (np.array,np.array,np.array) :
  ''' SVD is updated version of Eigen decomposition

  :param A: np.array - Matrix to be decomposed
  :param verify: bool - to print the verification of the function
  :return: (np.array,np.array,np.array) - Orthogonal matrix, Diagonal matrix and Transpose of Orthogonal matrix'''
  A_T_A = np.dot(A.T,A)
  eigen_values,eigen_vectors = np.linalg.eig(A_T_A)

  # sorting eigen vector in reverse order
  index = np.argsort(eigen_values)[::-1]
  eigen_values = eigen_values[index]
  eigen_vectors = eigen_vectors[:,index]

  # calculating the singular values
  singular_values = np.sqrt(np.abs(eigen_values))

  # diagonal matrix of singular values
  diagonal_matrix = np.diag(singular_values)

  # calculating the left singular values
  U = np.dot(A,eigen_vectors) / singular_values

  if verify:
    print("Verification")
    print(f"Given matrix A : \n{A}")
    print(f"Orthogonal matrix U : \n{U}")
    print(f"Diagonal matrix of singular values : \n{diagonal_matrix}")
    print(f"Transpose of Orthogonal matrix V : \n{eigen_vectors.T}")
    print(f"U @ Σ @ V : \n{np.allclose(A,np.dot(np.dot(U,diagonal_matrix),eigen_vectors.T))}")

  return U,diagonal_matrix,eigen_vectors.T


In [ ]:
A = np.array([[2, 1, 1, 0],
              [4, 3, 3, 1],
              [8, 7, 9, 5],
              [6, 7, 9, 8]])

SVD(A,True)

Verification
Given matrix A : 
[[2 1 1 0]
 [4 3 3 1]
 [8 7 9 5]
 [6 7 9 8]]
Orthogonal matrix U : 
[[ 0.09405951  0.37837471 -0.38179085 -0.83798636]
 [ 0.25637107  0.54175772 -0.58921648  0.54184536]
 [ 0.67402341  0.39256817  0.62495603 -0.0318219 ]
 [ 0.68638118 -0.63970414 -0.3413058  -0.05630135]]
Diagonal matrix of singular values : 
[[21.85847672  0.          0.          0.        ]
 [ 0.          3.45065757  0.          0.        ]
 [ 0.          0.          0.50598155  0.        ]
 [ 0.          0.          0.          0.2096204 ]]
Transpose of Orthogonal matrix V : 
[[ 0.49061414  0.4751477   0.59962157  0.41711679]
 [ 0.64512366  0.07931708 -0.06392284 -0.75725698]
 [-0.3332812  -0.32390239  0.79728567 -0.38515769]
 [-0.48170206  0.81426698 -0.02652189 -0.32284517]]
U @ Σ @ V : 
True


(array([[ 0.09405951,  0.37837471, -0.38179085, -0.83798636],
        [ 0.25637107,  0.54175772, -0.58921648,  0.54184536],
        [ 0.67402341,  0.39256817,  0.62495603, -0.0318219 ],
        [ 0.68638118, -0.63970414, -0.3413058 , -0.05630135]]),
 array([[21.85847672,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  3.45065757,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.50598155,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.2096204 ]]),
 array([[ 0.49061414,  0.4751477 ,  0.59962157,  0.41711679],
        [ 0.64512366,  0.07931708, -0.06392284, -0.75725698],
        [-0.3332812 , -0.32390239,  0.79728567, -0.38515769],
        [-0.48170206,  0.81426698, -0.02652189, -0.32284517]]))

# Matrix Approximation
Matrix approximation is converting bigger matrix into smaller matrix.
This concept is like `80-20 rule`


## Best K rank with SVD
We only select `k` elements from the SVD

In [3]:
def best_k_SVD(A: np.array , k : int) -> np.array:
  '''
  best_k_SVD is a matrix approximation function

  :param A: np.array - matrix
  :param k: int - k value
  :returns: np.array - approximated matrix'''
  U, Sigma, V_T = SVD(A)
  # selecting k elements
  U_k = U[:,:k]
  Sigma_k = Sigma[:k,:k]
  V_T_k = V_T[:k,:]
  # approx matrix of A
  A_k = np.dot(U_k,np.dot(Sigma_k,V_T_k))

  return A_k

In [5]:
A = np.array([[2, 1, 1, 0],
              [4, 3, 3, 1],
              [8, 7, 9, 5],
              [6, 7, 9, 8]])

approx_A = best_k_SVD(A,k=2)

approx_A

array([[ 1.85100171,  1.08046217,  1.14936014, -0.1311151 ],
       [ 3.95535059,  2.81094817,  3.24070931,  0.92184126],
       [ 8.10217572,  7.10785477,  8.74770842,  5.11963956],
       [ 5.93675919,  6.95367377,  9.13737379,  7.92967522]])